In [2]:
import itertools
import os

import jax
import jax.numpy as jnp
import numpy as np
from scipy import optimize

from vibrojet.keo import Gmat, pseudo, com
from vibrojet.potentials import h2co_AYTY
from vibrojet.taylor import deriv_list
from vibrojet.basis_utils import HermiteBasis, ContrBasis

jax.config.update("jax_enable_x64", True)

Equilibrium coordinates, around which the Taylor series expansions will be build

In [3]:
r0 = np.array([1.20337419, 1.10377465, 1.10377465, 2.1265833,  2.1265833,  np.pi])

Define internal to Cartesian coordinate transformation

In [4]:
masses = [12.0, 15.99491463, 1.00782505, 1.00782505]  # masses of C, O, H, H

ncoo = len(r0)


# @eckart(r0, masses)
@com(masses)
def internal_to_cartesian(internal_coords):
    rCO, rCH1, rCH2, aOCH1, aOCH2, tau = internal_coords
    xyz = [
        [0.0, 0.0, 0.0],
        [0.0, 0.0, rCO],
        [
            rCH1 * jnp.sin(aOCH1) * jnp.cos(tau * 0.5),
            -rCH1 * jnp.sin(aOCH1) * jnp.sin(tau * 0.5),
            rCH1 * jnp.cos(aOCH1),
        ],
        [
            rCH2 * jnp.sin(aOCH2) * jnp.cos(tau * 0.5),
            rCH2 * jnp.sin(aOCH2) * jnp.sin(tau * 0.5),
            rCH2 * jnp.cos(aOCH2),
        ],
    ]
    return jnp.array(xyz, dtype=jnp.float64)

Generate expansions for the KEO and PES

In [5]:
gmat_max_order = 4  # max total expansion order

gmat_terms = np.array(
    [
        elem
        for elem in itertools.product(
            *[range(0, gmat_max_order + 1) for _ in range(len(r0))]
        )
        if sum(elem) <= gmat_max_order
    ]
)
pseudo_terms = gmat_terms
print("max expansion order for G-matrix:", gmat_max_order)
print("number of expansion terms in G-matrix:", len(gmat_terms))

# expansion of G-matrix

gmat_file = f"_h2co_gmat_coefs_{gmat_max_order}.npy"
if os.path.exists(gmat_file):
    print(
        f"load G-matrix expansion coefs from file {gmat_file} (delete file to recompute coefficients)"
    )
    gmat_coefs = np.load(gmat_file)
else:
    gmat_coefs = deriv_list(
        lambda x: Gmat(x, masses, internal_to_cartesian), gmat_terms, r0, if_taylor=True
    )
    np.save(gmat_file, gmat_coefs)

# expansion of pseudopotential

pseudo_file = f"_h2co_pseudo_coefs_{gmat_max_order}.npy"
if os.path.exists(pseudo_file):
    print(
        f"load pseudopotential expansion coefs from file {pseudo_file} (delete file to recompute coefficients)"
    )
    pseudo_coefs = np.load(pseudo_file)
else:
    pseudo_coefs = deriv_list(
        lambda x: pseudo(x, masses, internal_to_cartesian),
        gmat_terms,
        r0,
        if_taylor=True,
    )
    np.save(pseudo_file, pseudo_coefs)

max expansion order for G-matrix: 4
number of expansion terms in G-matrix: 210
load G-matrix expansion coefs from file _h2co_gmat_coefs_4.npy (delete file to recompute coefficients)
load pseudopotential expansion coefs from file _h2co_pseudo_coefs_4.npy (delete file to recompute coefficients)


In [6]:
poten_max_order = 6

poten_terms = np.array(
    [
        elem
        for elem in itertools.product(
            *[range(0, poten_max_order + 1) for _ in range(len(r0))]
        )
        if sum(elem) <= poten_max_order
    ]
)
print("max expansion order for PES:", poten_max_order)
print("number of expansion terms in PES:", len(poten_terms))

poten_file = f"_h2co_poten_coefs_{poten_max_order}.npy"
if os.path.exists(poten_file):
    print(
        f"load potential expansion coefs from file {poten_file} (delete file to recompute coefficients)"
    )
    poten_coefs = np.load(poten_file)
else:
    poten_coefs = deriv_list(h2co_AYTY.poten, poten_terms, r0, if_taylor=True)
    np.save(poten_file, poten_coefs)

max expansion order for PES: 6
number of expansion terms in PES: 924
load potential expansion coefs from file _h2co_poten_coefs_6.npy (delete file to recompute coefficients)


Define linear mapping between coordinates of Hermite functions and internal coordinates

In [7]:
mask = gmat_terms != 0
ind0 = np.where(mask.sum(axis=1) == 0)[0][0]
mu = np.diag(gmat_coefs[ind0])[:ncoo]

mask = poten_terms != 0
ind2 = np.array(
    [
        np.where((mask.sum(axis=1) == 1) & (poten_terms[:, icoo] == 2))[0][0]
        for icoo in range(ncoo)
    ]
)
freq = poten_coefs[ind2] * 2

lin_a = jnp.sqrt(jnp.sqrt(mu / freq))
lin_b = r0

print("x->r linear mapping parameters 'a':", lin_a)
print("x->r linear mapping parameters 'b':", lin_b)

# x->r linear mapping function
x_to_r_map = lambda x, icoo: lin_a[icoo] * x + lin_b[icoo]

x->r linear mapping parameters 'a': [0.05265804 0.11094744 0.11094744 0.15432971 0.15432971 0.30023422]
x->r linear mapping parameters 'b': [1.20337419 1.10377465 1.10377465 2.1265833  2.1265833  3.14159265]


Define primitive Hermite basis

In [8]:
ncoo = len(r0)
nbas = [60] * ncoo
npoints = [80] * ncoo

p_r1, p_r2, p_r3, p_alpha1, p_alpha2, p_tau = [
    HermiteBasis(
        icoo,
        nbas[icoo],
        npoints[icoo],
        lambda x: x_to_r_map(x, icoo),
        lambda r: r - r0[icoo],
        lambda r: r - r0[icoo],
        gmat_terms[:, icoo],
        poten_terms[:, icoo],
        pseudo_terms[:, icoo],
    )
    for icoo in range(ncoo)
]

In [9]:
c_r1, c_r2, c_r3, c_alpha1, c_alpha2, c_tau = [
    ContrBasis(
        (icoo,),
        [p_r1, p_r2, p_r3, p_alpha1, p_alpha2, p_tau],
        lambda _: True,
        gmat_terms,
        gmat_coefs,
        poten_terms,
        poten_coefs,
        pseudo_terms,
        pseudo_coefs,
        emax=40000,
    )
    for icoo in range(ncoo)
]

e = c_r1.enr
print("Solutions for r1 mode:\n", f"zpe = {e[0]}\n", e - e[0])
e = c_r2.enr
print("\nSolutions for r2 mode:\n", f"zpe = {e[0]}\n", e - e[0])
e = c_r3.enr
print("\nSolutions for r3 mode:\n", f"zpe = {e[0]}\n", e - e[0])
e = c_alpha1.enr
print("Solutions for alpha1 mode:\n", f"zpe = {e[0]}\n", e - e[0])
e = c_alpha2.enr
print("Solutions for alpha2 mode:\n", f"zpe = {e[0]}\n", e - e[0])
e = c_tau.enr
print("Solutions for tau mode:\n", f"zpe = {e[0]}\n", e - e[0])

Solutions for r1 mode:
 zpe = 3485.3759139340295
 [    0.          1765.29002766  3507.43226225  5226.53555048
  6922.82790318  8596.69484134 10248.7224321  11879.74297453
 13490.87953515 15083.58347843 16659.65731608 18221.25451102
 19770.84935921 21311.17434189 22845.12885433 24375.67002037
 25905.70072634 27437.97015916 28974.9980777  30519.02772443
 32072.00623767 33635.58753608 35211.15230589]

Solutions for r2 mode:
 zpe = 3741.949778606593
 [    0.          2842.72544086  5577.70363256  8244.1752935
 10904.23318073 13618.33259052 16425.35291278 19343.20860607
 22377.71060586 25529.06386012 28795.09985856 32172.72899253
 35658.5861382 ]

Solutions for r3 mode:
 zpe = 3741.949778606594
 [    0.          2842.72544086  5577.70363256  8244.1752935
 10904.23318073 13618.33259052 16425.35291278 19343.20860607
 22377.71060586 25529.06386012 28795.09985856 32172.72899253
 35658.5861382 ]
Solutions for alpha1 mode:
 zpe = 3421.446436985066
 [    0.          1493.23803762  2986.26197249  

In [10]:
c_r2_r3 = ContrBasis(
    (1, 2),
    [c_r1, c_r2, c_r3, c_alpha1, c_alpha2, c_tau],
    lambda _: True,
    gmat_terms,
    gmat_coefs,
    poten_terms,
    poten_coefs,
    pseudo_terms,
    pseudo_coefs,
    emax=40000,
)

e = c_r2_r3.enr
print("Solutions for r2+r3 modes:\n", f"zpe = {e[0]}\n", e - e[0])

Solutions for r2+r3 modes:
 zpe = 4445.855851071909
 [    0.          2825.41421554  2868.35087459  5570.49386906
  5587.33838952  5711.76234059  8245.53674171  8250.62975973
  8410.00175177  8486.31785477 10906.57473368 10908.62030236
 11077.99528299 11126.41527899 11229.59292824 13612.75458275
 13614.85663716 13739.62541955 13777.0972272  13859.22674218
 13941.80183016 16370.07752734 16387.0046499  16437.22373825
 16485.91804358 16539.91307303 16594.9125869  16648.29245555
 19060.83942848 19146.35993374 19215.53212087 19273.34729546
 19326.36761492 19355.34116788 19420.37766581 19423.05102228
 21729.95062342 21867.1145912  21967.9197605  22063.71760877
 22091.98590258 22237.72839237 22238.8670587  22425.91439571
 22425.9295571  24413.81775833 24589.97119632 24731.53043103
 24798.19655516 24984.07803572 24987.90805983 25267.37345261
 25267.43043122 25569.00076965 25569.00125892 27126.75166132
 27340.31752723 27486.07347591 27666.59732035 27688.97798481
 28015.84515987 28016.29008445 2

In [11]:
c_alpha1_alpha2 = ContrBasis(
    (3, 4),
    [c_r1, c_r2, c_r3, c_alpha1, c_alpha2, c_tau],
    lambda _: True,
    gmat_terms,
    gmat_coefs,
    poten_terms,
    poten_coefs,
    pseudo_terms,
    pseudo_coefs,
    emax=40000,
)

e = c_alpha1_alpha2.enr
print("Solutions for alpha1+alpha2 modes:\n", f"zpe = {e[0]}\n", e - e[0])

Solutions for alpha1+alpha2 modes:
 zpe = 3675.729100999227
 [    0.          1265.64651398  1581.67814505  2534.68740381
  2843.57513911  3164.0481692   3807.86831111  4109.26635737
  4422.03453424  4746.58751073  5085.90201206  5379.53549752
  5684.20260852  6000.52445058  6328.75181749  6369.46483467
  6655.1272402   6951.37399773  7259.01754886  7578.52473877
  7659.19413074  7909.97012389  7936.74276507  8224.32465127
  8522.92577525  8833.2141784   8955.68672521  9155.49095065
  9225.03677589  9489.63916443  9503.77965243  9793.0552708
 10093.71559247 10259.49822953 10406.27304112 10520.61591336
 10730.84916103 10790.41004502 11067.11656242 11070.15357754
 11360.86389154 11571.14308684 11663.24718494 11824.03836921
 11977.64744461 12084.83148668 12303.98977283 12354.906505
 12635.4272492  12641.71253943 12891.09520933 12927.27539859
 13135.81448291 13230.9987647  13387.60442842 13546.75774411
 13647.93728828 13874.25998752 13918.10421955 14199.14434789
 14212.67965761 14219.78914

In [12]:
p_max = 8


def select_quanta(ind):
    n_r0, n_r12, n_a12, n_tau = ind
    cond = (
        n_r0 * 2 + int(np.ceil(n_r12 / 2)) * 2 + int(np.ceil(n_a12 / 2)) + n_tau
        <= p_max
    )
    return cond


sol = ContrBasis(
    (0, 1, 2, 3),
    [c_r1, c_r2_r3, c_alpha1_alpha2, c_tau],
    select_quanta,
    gmat_terms,
    gmat_coefs,
    poten_terms,
    poten_coefs,
    pseudo_terms,
    pseudo_coefs,
    emax=20000,
    store_me=False,
)

e = sol.enr
print("Solutions:\n", f"zpe = {e[0]}\n", e - e[0])

Solutions:
 zpe = 5803.016647281382
 [    0.          1175.07592449  1255.60130434  1516.5705387
  1751.9623785   2350.48568603  2430.12626388  2512.7946017
  2691.50279877  2743.18199867  2814.86420619  2855.52289735
  2920.67362192  3007.98411428  3038.78290444  3266.32301521
  3488.89648998  3524.56744728  3600.94718943  3686.7543806
  3775.16438489  3873.35682065  3918.01033303  3980.94211335
  3991.17207156  4016.96574643  4079.78566745  4093.14849939
  4108.3933533   4176.5494374   4218.76624073  4267.40925697
  4280.74325526  4355.39581706  4383.84201889  4449.32704157
  4531.74179993  4593.95402752  4628.59778535  4688.17200982
  4705.41466479  4730.44890674  4775.81153297  4781.4553803
  4858.67061069  4975.37784724  4977.52334734  5057.90879717
  5092.63501493  5133.17981396  5167.10212758  5194.56422528
  5196.05120601  5197.10010219  5273.31296756  5280.31783778
  5296.2924682   5335.20396983  5372.39364902  5379.29633416
  5419.9129824   5439.31039787  5464.51419672  5487.